# NBA Data Scrape

In [29]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import logging
import re
import time
import datetime
from datetime import date
from dateutil.parser import parse

In [30]:
# Setup the csv writer
nbafile = open("NBA_data/NBA_Season_avgs_2008.csv", "w", newline='', encoding='utf-8')
nbawriter = csv.writer(nbafile)
# set headings
nbawriter.writerow(['RK', 
                    'Team', 
                    'Year', 
                    'G', 
                    'W', 
                    'L', 
                    'Conf', 
                    'MP', 
                    'FG', 
                    'FGA', 
                    'FG%', 
                    '3P',
                    '3PA', 
                    '3P%', 
                    '2P', 
                    '2PA', 
                    '2P%', 
                    'FT', 
                    'FTA', 
                    'FT%', 
                    'ORB', 
                    'DRB', 
                    'AST', 
                    'STL', 
                    'BLK', 
                    'TOV', 
                    'PF', 
                    'PTS'])

103

## Select Start and End Year

In [37]:
# Define Year for URL
start_year = 2005
end_year = 2006

In [38]:
# Create model to scrape data

class NBAscrapy:
    runcount = 1
    rk = 1
    year = 2005
    
    # __init__ required to initialize the scrape tool
    def __init__(self):
        self.main()

    # starting URL according to start year above
    def main(self):
        url = "https://www.basketball-reference.com/leagues/NBA_"+str(start_year)+".html"
        self.getyearteamdata(url)

    # build data for per-year
    def getyearteamdata(self, url):
        
        # running through each table by rank since website uses rank in each table used in ascending order
        # rank will reset to 1 once we move to a new year
        self.rk = 1
        print("--------------year: " + str(self.year) + " |" + " row: " + str(self.runcount) + "-----------------")
        
        # get data, total_pages
        page = requests.get(url)
        page.encoding = "UTF-8"
        pagecontent = str(page.content)
        pagecontent = pagecontent.replace('<!--',"")
        pagecontent = pagecontent.replace("-->", "")
        pagecontent = pagecontent.replace("\n", "")

        # print(pagecontent)
        soup = BeautifulSoup(pagecontent, 'html.parser')
        soup.encode('utf-8')

        # format material for page.json()
        
        # data will be pulled from 'Team Per Game Stats' table (avg game data)
        Team = ""
        Year = self.year
        G = ""
        MP = ""
        FG = ""
        FGA = ""
        FGpercent = ""
        ThreeP = ""
        ThreePA = ""
        ThreePpercent = ""
        TwoP = ""
        TwoPA = ""
        TwoPpercent = ""
        FT = ""
        FTA = ""
        FTpercent = ""
        ORB = ""
        DRB = ""
        TRB = ""
        AST = ""
        STL = ""
        BLK = ""
        TOV = ""
        PF = ""
        PTS = ""
        
        # data will be pulled from 'Conference Standings' table
        W = ""
        L = ""
        Conf = ""
        E_W_teams = []

        # for testing we wanted to split between East and West
        # This required pulling from the 'Conference Standings' table
        
        # If statement to Eastern Conference
        if soup.find('table', {'id':'divs_standings_E'}) is not None:
            if soup.find('table', {'id':'divs_standings_E'}).findAll('tr', {'class': 'full_table'}) is not None:
                E_team_trs = soup.find('table', {'id':'divs_standings_E'}).findAll('tr', {'class': 'full_table'})
                nI = 0
                
                # Loop through Eastern Conference, collect wins, lossess
                for E_team_tr in E_team_trs:
                    E_team = {
                        'index': 'East', # Index the data by 'East' Conference
                        'teamname': E_team_tr.find('a').text.strip(),
                        'W': E_team_tr.find('td',{'data-stat': 'wins'}).text.strip(),
                        'L': E_team_tr.find('td',{'data-stat': 'losses'}).text.strip()
                    }
                    E_W_teams.append(E_team.copy())
                    nI += 1

        # If statement to Western Conference
        if soup.find('table', {'id':'divs_standings_W'}) is not None:
            if soup.find('table', {'id':'divs_standings_W'}).findAll('tr', {'class': 'full_table'}) is not None:
                W_team_trs = soup.find('table', {'id':'divs_standings_W'}).findAll('tr', {'class': 'full_table'})
                nI = 0
                
                # Loop through Western Conference, collect wins, lossess
                for W_team_tr in W_team_trs:
                    W_team = {
                        'index': 'West', # Index the data by 'West' Conference
                        'teamname': W_team_tr.find('a').text.strip(),
                        'W': W_team_tr.find('td',{'data-stat': 'wins'}).text.strip(),
                        'L': W_team_tr.find('td',{'data-stat': 'losses'}).text.strip()
                    }
                    E_W_teams.append(W_team.copy())
                    nI += 1
                    
        # If statement to collect data from league "Team Per Game Stats" table            
        if soup.find('table', {'id':'per_game-team'}) is not None:
            if soup.find('table', {'id':'per_game-team'}).findAll('tr') is not None:
                Team_stats_trs = soup.find('table', {'id':'per_game-team'}).findAll('tr')
                #print(Team_stats_trs)
                for Team_stats_tr in Team_stats_trs:
                    thhead = Team_stats_tr.find('th',{'scope': 'row'})
                    sheet = []
                    if thhead is not None:
                        Team = Team_stats_tr.find('td',{'data-stat': 'team'}).text.strip()
                        if Team != "League Average":
                            G = Team_stats_tr.find('td',{'data-stat': 'g'}).text.strip()
                            MP = Team_stats_tr.find('td',{'data-stat': 'mp'}).text.strip()
                            FG = Team_stats_tr.find('td',{'data-stat': 'fg'}).text.strip()
                            FGA = Team_stats_tr.find('td',{'data-stat': 'fga'}).text.strip()
                            FGpercent = Team_stats_tr.find('td',{'data-stat': 'fg_pct'}).text.strip()
                            ThreeP = Team_stats_tr.find('td',{'data-stat': 'fg3'}).text.strip()
                            ThreePA = Team_stats_tr.find('td',{'data-stat': 'fg3a'}).text.strip()
                            ThreePpercent = Team_stats_tr.find('td',{'data-stat': 'fg3_pct'}).text.strip()
                            TwoP = Team_stats_tr.find('td',{'data-stat': 'fg2'}).text.strip()
                            TwoPA = Team_stats_tr.find('td',{'data-stat': 'fg2a'}).text.strip()
                            TwoPpercent = Team_stats_tr.find('td',{'data-stat': 'fg2_pct'}).text.strip()
                            FT = Team_stats_tr.find('td',{'data-stat': 'ft'}).text.strip()
                            FTA = Team_stats_tr.find('td',{'data-stat': 'fta'}).text.strip()
                            FTpercent = Team_stats_tr.find('td',{'data-stat': 'ft_pct'}).text.strip()
                            ORB = Team_stats_tr.find('td',{'data-stat': 'orb'}).text.strip()
                            DRB = Team_stats_tr.find('td',{'data-stat': 'drb'}).text.strip()
                            TRB = Team_stats_tr.find('td',{'data-stat': 'trb'}).text.strip()
                            AST = Team_stats_tr.find('td',{'data-stat': 'ast'}).text.strip()
                            STL = Team_stats_tr.find('td',{'data-stat': 'stl'}).text.strip()
                            BLK = Team_stats_tr.find('td',{'data-stat': 'blk'}).text.strip()
                            TOV = Team_stats_tr.find('td',{'data-stat': 'tov'}).text.strip()
                            PF = Team_stats_tr.find('td',{'data-stat': 'pf'}).text.strip()
                            PTS = Team_stats_tr.find('td',{'data-stat': 'pts'}).text.strip()

                            # Confirm the Win, Loss, and Conference
                            e_w_confirm = self.check_E_W(E_W_teams, Team)
                            if e_w_confirm != "error":
                                W = e_w_confirm['W']
                                L = e_w_confirm['L']
                                Conf = e_w_confirm['index']
                                
                            # append ALL team data
                            sheet.append(self.rk)
                            sheet.append(Team)
                            sheet.append(Year)
                            sheet.append(G)

                            sheet.append(W)
                            sheet.append(L)
                            sheet.append(Conf)

                            sheet.append(MP)
                            sheet.append(FG)
                            sheet.append(FGA)
                            sheet.append(FGpercent)
                            sheet.append(ThreeP)
                            sheet.append(ThreePA)
                            sheet.append(ThreePpercent)
                            sheet.append(TwoP)
                            sheet.append(TwoPA)
                            sheet.append(TwoPpercent)
                            sheet.append(FT)
                            sheet.append(FTA)
                            sheet.append(FTpercent)
                            sheet.append(ORB)
                            sheet.append(DRB)
                            sheet.append(AST)
                            sheet.append(STL)
                            sheet.append(BLK)
                            sheet.append(TOV)
                            sheet.append(PF)
                            sheet.append(PTS)
                            self.rk += 1
                            print(sheet)
                            nbawriter.writerow(sheet)
                            print("--------------------------------------------------------")
                            print(" ")
                            

        print("--------------------------------------------------------")
        print(" ")

        # move to next page to scrape if under year boundries
        if self.year <= (end_year-1):
            self.runcount += 1
            self.year += 1
            url = "https://www.basketball-reference.com/leagues/NBA_" + str(self.year) + ".html"
            self.getyearteamdata(url)

    def check_E_W(self, E_W_teams, teamname):
        t_name = "error"
        for E_W_team in E_W_teams:
            if E_W_team['teamname'] == teamname.replace("*",""):
                t_name = E_W_team
                break
        return t_name
NBAscrapy()

--------------year: 2005 | row: 1-----------------
[1, 'Phoenix Suns*', 2005, '82', '62', '20', 'West', '241.2', '40.9', '85.6', '.477', '9.7', '24.7', '.393', '31.2', '60.9', '.512', '19.0', '25.4', '.748', '11.8', '32.3', '23.5', '7.0', '5.5', '13.7', '19.1', '110.4']
--------------------------------------------------------
 
[2, 'Sacramento Kings*', 2005, '82', '50', '32', 'West', '242.1', '39.1', '85.1', '.459', '6.4', '17.0', '.374', '32.7', '68.1', '.480', '19.2', '24.4', '.787', '12.5', '29.9', '24.5', '8.2', '3.9', '13.1', '20.5', '103.7']
--------------------------------------------------------
 
[3, 'Dallas Mavericks*', 2005, '82', '58', '24', 'West', '240.6', '37.3', '81.6', '.457', '5.6', '15.5', '.364', '31.6', '66.1', '.479', '22.3', '28.2', '.789', '12.1', '30.9', '19.6', '8.6', '5.6', '13.4', '22.3', '102.5']
--------------------------------------------------------
 
[4, 'Miami Heat*', 2005, '82', '59', '23', 'East', '243.7', '37.8', '77.7', '.486', '5.8', '15.4', '.377

[1, 'Phoenix Suns*', 2006, '82', '54', '28', 'West', '243.4', '41.8', '87.4', '.479', '10.2', '25.6', '.399', '31.6', '61.8', '.511', '14.5', '18.0', '.806', '9.5', '32.3', '26.6', '6.7', '5.0', '13.3', '20.5', '108.4']
--------------------------------------------------------
 
[2, 'Seattle SuperSonics', 2006, '82', '35', '47', 'West', '241.8', '37.5', '81.8', '.459', '7.4', '19.9', '.371', '30.1', '62.0', '.487', '20.1', '25.7', '.785', '12.4', '27.2', '20.7', '7.6', '3.7', '14.7', '23.6', '102.6']
--------------------------------------------------------
 
[3, 'Washington Wizards*', 2006, '82', '42', '40', 'East', '241.8', '36.3', '81.2', '.447', '6.1', '17.0', '.357', '30.2', '64.2', '.471', '23.0', '30.4', '.757', '12.6', '28.6', '18.6', '8.0', '4.1', '13.9', '22.6', '101.7']
--------------------------------------------------------
 
[4, 'Toronto Raptors', 2006, '82', '27', '55', 'East', '243.4', '36.7', '81.0', '.454', '7.4', '19.8', '.375', '29.3', '61.2', '.479', '20.2', '25.5', 